# Introduction to ensemble methods

In this practical, you will construct your own bagging ensemble, to boost performance on a binary classification problem.


## The problem

Users of your service sometimes close their accounts, even though they have been with you for a while. It would be useful if you could predict which ones are likely to do this, before it happens, so you can take action to retain their custom.

You have access to 20 different features for each user, based on data such as how often they log in, how many purchases they make, etc. The aim is to see if you can predict whether a user will end up closing their account or not.

## Getting started

First, load the data from `account_data.csv` using `pandas` into a DataFrame called `data`. 

Find out how many users left the service by examining the `closed_account` column. 

*Hint: use the `.value_counts()` method*

In [33]:
import pandas as pd

# Your code here...
data = pd.read_csv("account_data.csv")
data.head()

data['closed_account'].value_counts()

closed_account
1    102
0     98
Name: count, dtype: int64

## Setting up for classification - I

Before proceeding, we will split the data into training and testing subsets.

Create two variables, `X` and `y`, from the `data` DataFrame.

`X` should be all the columns that you want to use in your prediction (ie. all except for `closed_account`)

`y` should be the column that you are trying to predict (ie. `closed_account`)

These are the conventional variable names used in machine learning to represent your predicting features (`X`) and the values you are trying to predict (`y`).

In [34]:
# Your code here...
X = data.copy().drop(columns = 'closed_account')
y = data['closed_account']

## Setting up for classification - II

Models will have access to the training set in order to learn, but we will evaluate them on the unseen test set.

Use `train_test_split` from `sklearn.model_selection` to split the data up. 

This function takes your `X` and `y` (our 20 monthly data points, and the closed/not closed label) and splits them up. It returns four lists - one `X` for each of train/test and the same for `y`.

The default gives a ratio of 3:1 train/test split.

Call the new variables `X_train`, `X_test`, `y_train`, and `y_test`.

(Set `random_state` to `5`, so that results are the same every time!)

In [35]:
from sklearn.model_selection import train_test_split

# Your code here...
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=5)

## Single model performance - I

Before trying an ensemble, we need some baselines in order to determine whether the ensemble method actually improved anything.

We will use a simple Naive Bayes classifier here with default parameters.

Import the required class and instantiate it with the variable name `model`.


In [36]:
from sklearn.naive_bayes import GaussianNB

# Your code here...
model = GaussianNB()

## Single model performance - II

Use the `.fit()` method of the model to train it on the training subset of the data.

Use the `.score()` method the same way, using the same data, to see how well it performs on the data it learned from.

Use `model.predict()` to generate predictions for `X_train` and compare these predictions to the ground truth (`y_train`) using a classification report.

In [37]:
from sklearn.metrics import classification_report

# Your code here...
model.fit(X_train, y_train)

# accuracy score
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, model.predict(X_train)))
print(classification_report(y_train, model.predict(X_train)))

0.6733333333333333
              precision    recall  f1-score   support

           0       0.65      0.60      0.63        68
           1       0.69      0.73      0.71        82

    accuracy                           0.67       150
   macro avg       0.67      0.67      0.67       150
weighted avg       0.67      0.67      0.67       150



## Single model performance - III

Repeat the above scoring process but now use the unseen test data.

Do you expect accuracy to change? If so, in what direction?

What are your thoughts on how well the model performs on the two classes?

In [38]:
# Your code here...
print(accuracy_score(y_test, model.predict(X_test)))
print(classification_report(y_test, model.predict(X_test)))

0.44
              precision    recall  f1-score   support

           0       0.54      0.43      0.48        30
           1       0.35      0.45      0.39        20

    accuracy                           0.44        50
   macro avg       0.44      0.44      0.44        50
weighted avg       0.46      0.44      0.45        50



## Bagging ensemble - I

Now let's see what effect a simple bagging ensemble has.

`sklearn` has a useful utility class that will handle this for us: `sklearn.ensemble.BaggingClassifier`

This has several parameters which can be tuned but the main ones are:

`base_estimator` - the model to use

`n_estimators` - how many models to use. Default is 10.

`random_state` - ensures the same results every time

Create a bagging classifier named `ensemble`, using `GaussianNB()` as the `base_estimator`. Keep `n_estimators` at 10 and set `random_state` to 5.

In [39]:
from sklearn.ensemble import BaggingClassifier

# Your code here...
ensemble = BaggingClassifier(estimator=GaussianNB(), n_estimators=10, random_state=5)
ensemble.fit(X_train, y_train)

BaggingClassifier(estimator=GaussianNB(), random_state=5)

## Bagging ensemble - II

Once you have created this meta-model, it works exactly the same way as any model in `sklearn` - it has  methods like `.fit()` and `.score()` and `.predict()`.

Repeat the training/scoring/classification report process from the single model approach you did previously.

Train only on the training data and test only on the test data.

What do you observe now?

In [40]:
# Your code here...
print(accuracy_score(y_train, ensemble.predict(X_train)))
print(accuracy_score(y_test, ensemble.predict(X_test)))

print(classification_report(y_train, ensemble.predict(X_train)))
print(classification_report(y_test, ensemble.predict(X_test)))

0.66
0.58
              precision    recall  f1-score   support

           0       0.63      0.60      0.62        68
           1       0.68      0.71      0.69        82

    accuracy                           0.66       150
   macro avg       0.66      0.66      0.66       150
weighted avg       0.66      0.66      0.66       150

              precision    recall  f1-score   support

           0       0.70      0.53      0.60        30
           1       0.48      0.65      0.55        20

    accuracy                           0.58        50
   macro avg       0.59      0.59      0.58        50
weighted avg       0.61      0.58      0.58        50



## Conclusions

Using an ensemble of methods improved performance in a binary classification task by a good margin, though performance was still generally quite low.

This may be because the signal to noise ratio in the data is low (we have used synthetic data), meaning the model needs to make predictions with very little information. Even so, what could you investigate in order to improve performance on this task?

In [41]:
# Your code below


GridSearchCV(estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         max_features=0.5),
             param_grid={'estimator__max_depth': [1, 2, 3, 4, 5],
                         'max_samples': [0.05, 0.1, 0.2, 0.5],
                         'n_estimators': [10, 20, 30]},
             scoring='f1')

In [45]:
clf

print(accuracy_score(y_train, clf.predict(X_train)))
print(accuracy_score(y_test, clf.predict(X_test)))

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test)))

0.6
0.56
              precision    recall  f1-score   support

           0       0.55      0.62      0.58        68
           1       0.65      0.59      0.62        82

    accuracy                           0.60       150
   macro avg       0.60      0.60      0.60       150
weighted avg       0.61      0.60      0.60       150

              precision    recall  f1-score   support

           0       0.65      0.57      0.61        30
           1       0.46      0.55      0.50        20

    accuracy                           0.56        50
   macro avg       0.56      0.56      0.55        50
weighted avg       0.58      0.56      0.56        50

